In [1]:
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize 
import matplotlib
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [2]:
# After variable change C = F * C, xi = F * xi and lambda = F * lambda for both OFF and ON model
xi_values = np.arange(0,5,0.5)
dr = 0.05
phi_values = np.arange(dr,8,0.5)
C_values = np.arange(0,0.6,0.05)

dirac = 1/dr
phi_max = 10
n = int(phi_max/dr)

def CC_on(phi):
    return 0.5 * phi / (1 + phi)

def CC_off(phi):
    return 0.5 * 1 / (1 + phi)

eigenvalues_off = []
eigenvalues_on = []
for CC in C_values:
    save_eigenvalues_off = []
    save_eigenvalues_on = []
    for phi in phi_values:
        R = int(phi/dr)
        max_eigen_values_off = []
        max_eigen_values_on = []
        for xi in xi_values:

            # Matrices OFF
            A_off = np.zeros((n,n), dtype='cfloat')
            B_off = np.zeros((n,n), dtype='cfloat')
            C_off = np.zeros((n,n), dtype='cfloat')
            D_off = np.zeros((n,n), dtype='cfloat')

            # Matrices ON
            A_on = np.zeros((n,n), dtype='cfloat')
            B_on = np.zeros((n,n), dtype='cfloat')
            C_on = np.zeros((n,n), dtype='cfloat')
            D_on = np.zeros((n,n), dtype='cfloat')

            # Renormalisation for the sum of exponentials
            renorm = np.sum(dr * np.exp(-(np.arange(R,n) - R) * dr))

            for i in range(1,n):
                # Haut gauche
                A_off[i,i] = 1j*xi + 1/dr*(i<n-1)
                A_off[i,i-1] = -1/dr
                if i >= R:
                    A_off[i,i] += 1
                    for j in range(n):
                        A_off[i,j] += CC * np.exp(-(i - R) * dr) * dr / renorm
                        B_off[i,j] += CC * np.exp(-(i - R) * dr) * dr / renorm

                A_on[i,i] = 1j*xi + 1/dr*(i<n-1)
                A_on[i,i-1] = -1/dr
                if i == R:
                    A_on[i,i] += 1
                    for j in range(n):
                        A_on[i,j] += CC * dirac * dr
                        B_on[i,j] += CC * dirac * dr
                if i > R:
                    A_on[i,i] += 1

               # Bas droite
                D_off[i,i] = -1j*xi + 1/dr*(i<n-1)
                D_off[i,i-1] = -1/dr
                if i >= R:
                    D_off[i,i] += 1
                    for j in range(i,n):
                        D_off[i,j] += CC * np.exp(-(i - R) * dr) * dr / renorm
                        C_off[i,j] += CC * np.exp(-(i - R) * dr) * dr / renorm

                D_on[i,i] = -1j*xi + 1/dr*(i<n-1)
                D_on[i,i-1] = -1/dr
                if i == R:
                    D_on[i,i] += 1
                    for j in range(n):
                        D_on[i,j] += CC * dirac * dr
                        C_on[i,j] += CC * dirac * dr
                if i > R:
                    D_on[i,i] += 1

            # First line OFF 
            A_off[0,0] = 1j*xi + 1/dr
            A_off[0,:] -= CC

            B_off[0,:] -= CC
            B_off[0,R:] -= 1

            C_off[0,:] -= CC
            C_off[0,R:] -= 1

            D_off[0,0] = -1j*xi + 1/dr
            D_off[0,:] -= CC

            # First line ON
            A_on[0,0] = 1j*xi + 1/dr
            A_on[0,:] -= CC

            B_on[0,:] -= CC
            B_on[0,R:] -= 1

            C_on[0,:] = -CC
            C_on[0,R:] -= 1

            D_on[0,0] = -1j*xi + 1/dr
            D_on[0,:] -= CC

            # Concatenate OFF matrices
            A = np.concatenate((A_off,B_off),axis = 1)
            B = np.concatenate((C_off,D_off),axis = 1)
            M_off = np.concatenate((A,B),axis=0)

            # Concatenate ON matrices
            A = np.concatenate((A_on,B_on),axis = 1)
            B = np.concatenate((C_on,D_on),axis = 1)
            M_on = np.concatenate((A,B),axis=0)

            max_eigen_values_off.append(np.max(LA.eig(-M_off)[0].real))
            max_eigen_values_on.append(np.max(LA.eig(-M_on)[0].real))

        save_eigenvalues_off.append(np.max(max_eigen_values_off))
        save_eigenvalues_on.append(np.max(max_eigen_values_on))
        
    eigenvalues_off.append(np.array(save_eigenvalues_off))
    eigenvalues_on.append(np.array(save_eigenvalues_on))
    
eigenvalues_off_nan = np.array(eigenvalues_off_nan)
eigenvalues_on_nan = np.array(eigenvalues_on_nan)

NameError: name 'eigenvalues_off_nan' is not defined

In [ ]:
def forceAspect(ax,aspect):
    im = ax.get_images()
    extent =  im[0].get_extent()
    ax.set_aspect(abs((extent[1]-extent[0])/(extent[3]-extent[2]))/aspect)

# replace <10e-12 value by nan
eigenvalues_off_nan = eigenvalues_off.copy()
eigenvalues_on_nan = eigenvalues_on.copy()
eigenvalues_off_nan[eigenvalues_off_nan<10e-12] = np.nan
eigenvalues_on_nan[eigenvalues_on_nan<10e-12] = np.nan

labelsize = 20
cmap=plt.get_cmap('plasma_r')
x_min = phi_values[0]
x_max = phi_values[-1]
y_min = C_values[0]
y_max = C_values[-1]

fig = plt.figure(figsize=(12,10))
ax = fig.add_subplot(111)
#ax.hist2d(x=gamma_values, y=xi_values, bins=[gamma_values.shape[0],xi_values.shape[0]], weights=eigenvalues_off)
im = plt.imshow(eigenvalues_off_nan, cmap=cmap, extent = [x_min, x_max, y_min, y_max], aspect='auto', origin='lower')
ax.plot(phi_values,CC_on(phi_values),color='lime',linewidth=4,label="$\~C_{on}$")
ax.plot(phi_values,CC_off(phi_values),color='pink',linewidth=4,label="$\~C_{off}$")
ax.legend(loc=3,fontsize=labelsize)
# forceAspect(ax,aspect=1.0)
# ax.set_title('Eigenvalues',fontsize=labelsize)
ax.set_xlabel('S', fontsize = labelsize)
#\overset{\sim}{C}
ax.set_ylabel('$\~C$', fontsize = labelsize)
ax.tick_params(axis='both', which='both', labelsize=labelsize)
v1 = np.linspace(np.array(eigenvalues_off).min(), np.array(eigenvalues_off).max(), 8, endpoint=True)
cb = plt.colorbar(ticks=v1)
cb.ax.set_yticklabels(["{:4.2f}".format(i) for i in v1], fontsize='15')
cb.set_label("$\lambda$",fontsize=fontsize)
# fig.savefig("../../../../Latex/Syntheses_these/Paper_model_myxo/images/spectre_off_on_with_C_curves", bbox_inches='tight')

#divider = make_axes_locatable(ax)
#cax = divider.append_axes("right", size="5%", pad=0.05)
#cbar = fig.colorbar(im, cax=cax)
#cbar.ax.set_ylabel('Density')
#cbar.ax.tick_params(labelsize=labelsize)
#cbar.ax.yaxis.get_offset_text().set(size=labelsize)
#ax_c = cbar.ax
#text = ax_c.yaxis.label
#font = matplotlib.font_manager.FontProperties(size=labelsize)
#text.set_font_properties(font)


fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
#ax.hist2d(x=gamma_values, y=xi_values, bins=[gamma_values.shape[0],xi_values.shape[0]], weights=eigenvalues_off)
im_off = plt.imshow(eigenvalues_off_nan, cmap=cmap, extent = [x_min, x_max, y_min, y_max], aspect='auto', origin='lower')
#ax.plot(phi_values,CC_on(phi_values),color='white',linewidth=4,label="$\~C_{on}$")
#ax.plot(phi_values,CC_off(phi_values),color='pink',linewidth=4,label="$\~C_{off}$")
#ax.legend(loc=3,fontsize=labelsize)
# forceAspect(ax,aspect=1.0)
ax.set_title('Eigenvalues OFF',fontsize=labelsize)
ax.set_xlabel('$S_{off}$', fontsize = labelsize)
#\overset{\sim}{C}
ax.set_ylabel('$\~C_{off}$', fontsize = labelsize)
ax.tick_params(axis='both', which='both', labelsize=labelsize)
v1 = np.linspace(np.array(eigenvalues_off).min(), np.array(eigenvalues_off).max(), 8, endpoint=True)
cb = plt.colorbar(ticks=v1)
cb.ax.set_yticklabels(["{:4.2f}".format(i) for i in v1], fontsize='15')
cb.set_label("$\lambda$",fontsize=fontsize)
# fig.savefig("../../../../Latex/Syntheses_these/Paper_model_myxo/images/spectre_off", bbox_inches='tight')

#divider = make_axes_locatable(ax)
#cax = divider.append_axes("right", size="5%", pad=0.05)
#cbar = fig.colorbar(im, cax=cax)
#cbar.ax.set_ylabel('Density')
#cbar.ax.tick_params(labelsize=labelsize)
#cbar.ax.yaxis.get_offset_text().set(size=labelsize)
#ax_c = cbar.ax
#text = ax_c.yaxis.label
#font = matplotlib.font_manager.FontProperties(size=labelsize)
#text.set_font_properties(font)

fontsize = 15

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
#ax.hist2d(x=gamma_values, y=xi_values, bins=[gamma_values.shape[0],xi_values.shape[0]], weights=eigenvalues_off)
im_on = plt.imshow(eigenvalues_on_nan, cmap=cmap, extent = [x_min, x_max, y_min, y_max], aspect='auto', origin='lower')
# forceAspect(ax,aspect=1)
ax.set_title('Eigenvalues ON',fontsize=labelsize)
ax.set_xlabel('$S_{on}$', fontsize = labelsize)
ax.set_ylabel('$\~C_{on}$', fontsize = labelsize)
ax.tick_params(axis='both', which='both', labelsize=labelsize)
v1 = np.linspace(np.array(eigenvalues_off).min(), np.array(eigenvalues_off).max(), 8, endpoint=True)
cb = plt.colorbar(ticks=v1)
cb.ax.set_yticklabels(["{:4.2f}".format(i) for i in v1], fontsize=fontsize)
cb.set_label("$\lambda$",fontsize=fontsize)
# fig.savefig("../../../../Latex/Syntheses_these/Paper_model_myxo/images/spectre_on", bbox_inches='tight')

#divider = make_axes_locatable(ax)
#cax = divider.append_axes("right", size="5%", pad=0.05)
#cbar = fig.colorbar(im, cax=cax)
#cbar.ax.set_ylabel('Density')
#cbar.ax.tick_params(labelsize=labelsize)
#cbar.ax.yaxis.get_offset_text().set(size=labelsize)
#ax_c = cbar.ax
#text = ax_c.yaxis.label
#font = matplotlib.font_manager.FontProperties(size=labelsize)
#text.set_font_properties(font)


fontsize = 15

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
#ax.hist2d(x=gamma_values, y=xi_values, bins=[gamma_values.shape[0],xi_values.shape[0]], weights=eigenvalues_off)
im_on = plt.imshow(eigenvalues_on_nan, cmap=cmap, extent = [x_min, x_max, y_min, y_max], aspect='auto', origin='lower')
# forceAspect(ax,aspect=1)
ax.set_xlabel('$S$', fontsize = labelsize)
ax.set_ylabel('$\~C$', fontsize = labelsize)
ax.tick_params(axis='both', which='both', labelsize=labelsize)
v1 = np.linspace(np.array(eigenvalues_off).min(), np.array(eigenvalues_off).max(), 8, endpoint=True)
cb = plt.colorbar(ticks=v1)
cb.ax.set_yticklabels(["{:4.2f}".format(i) for i in v1], fontsize=fontsize)
cb.set_label("$\lambda$",fontsize=fontsize)
# fig.savefig("../../../../Latex/Syntheses_these/Paper_model_myxo/images/spectre_on_off", bbox_inches='tight')